In [14]:
import numpy as np
from factor import *

# Belief Propagation

___
$\def\abs#1{\left\lvert #1 \right\rvert}
\def\Set#1{\left\{ #1 \right\}}
\def\mc#1{\mathcal{#1}}
\def\M#1{\boldsymbol{#1}}
\def\R#1{\mathsf{#1}}
\def\RM#1{\boldsymbol{\mathsf{#1}}}
\def\op#1{\operatorname{#1}}
\def\E{\op{E}}
\def\d{\mathrm{\mathstrut d}}$

The joint probability distribution of a factor graph of $N$ variables with $M$ functions can be found as follows:
$$P(x_1, ...,x_n) = Z^{-1} \prod_{k=1}^{M} \psi_k(x_k)$$

Denote $\tilde{q}_u (i)$ as the $i$-th entry of vector $\tilde{\M{q}}_u$ which is the probability that node $u$ in the class $i$. $\tilde{p}_u (i)$ and $\tilde{H}_{uv} (i,j)$ have the similar meaning.

Sum-Product Massage Passing

$$m_{u \to v}(i) = \sum_{j} \tilde{H}_{uv} (j,i) \prod_{k \in \mc{N}(u) ∖ v} m_{k \to u}(j) $$

Inference of marginal via massage
$$\tilde{p}_u (i) = \frac{1}{Z_u} \tilde{q}_u (i) \prod_{k \in \mc{N}(u)} m_{k \to u} (i)$$

## Factor Product and Distributions

A factor object is represented with variable name and a numpy array-like distributions. E.g. $P(b)$

In [16]:
phi_2 = factor(['b'], np.array([0.3,0.7]))

In [2]:
def factor_product(x, y):
    if x.is_none() or y.is_none():
        raise Exception('One of the factors is None')
    
    xy, xy_in_x_ind, xy_in_y_ind = np.intersect1d(x.get_variables(), y.get_variables(), return_indices=True)
    
    if xy.size == 0:
        raise Exception('Factors do not have common variables')
    
    if not np.all(x.get_shape()[xy_in_x_ind] == y.get_shape()[xy_in_y_ind]):
        raise Exception('Common variables have different order')
    
    x_not_in_y = np.setdiff1d(x.get_variables(), y.get_variables(), assume_unique=True)
    y_not_in_x = np.setdiff1d(y.get_variables(), x.get_variables(), assume_unique=True)
    
    x_mask = np.isin(x.get_variables(), xy, invert=True)
    y_mask = np.isin(y.get_variables(), xy, invert=True)
    
    x_ind = np.array([-1]*len(x.get_variables()), dtype=int)
    y_ind = np.array([-1]*len(y.get_variables()), dtype=int)
    
    x_ind[x_mask] = np.arange(np.sum(x_mask))
    y_ind[y_mask] = np.arange(np.sum(y_mask)) + np.sum(np.invert(y_mask))
    
    x_ind[xy_in_x_ind] = np.arange(len(xy)) + np.sum(x_mask)
    y_ind[xy_in_y_ind] = np.arange(len(xy))
    
    x_distribution = np.moveaxis(x.get_distribution(), range(len(x_ind)), x_ind)
    y_distribution = np.moveaxis(y.get_distribution(), range(len(y_ind)), y_ind)
                
    res_distribution =   x_distribution[tuple([slice(None)]*len(x.get_variables())+[None]*len(y_not_in_x))] \
                       * y_distribution[tuple([None]*len(x_not_in_y)+[slice(None)])]
    
    return factor(list(x_not_in_y)+list(xy)+list(y_not_in_x), res_distribution)

Consider the formula $P(a,b) = P(a \mid b)P(b)$

In [7]:
phi_1 = factor(['a','b'], np.array([[0.3,0.8],[0.2,0.1],[0.5,0.1]]))

phi_3 = factor_product(phi_1, phi_2)
phi_3.get_distribution()

array([[0.09, 0.56],
       [0.06, 0.07],
       [0.15, 0.07]])

Compute the marginalization of a given joint distribution.

In [8]:
def factor_marginalization(x, variables):
    variables = np.array(variables)
    
    if x.is_none():
        raise Exception('Factor is None')
    
    if not np.all(np.in1d(variables, x.get_variables())):
        raise Exception('Factor do not contain given variables')
    
    res_variables    = np.setdiff1d(x.get_variables(), variables, assume_unique=True)
    res_distribution = np.sum(x.get_distribution(),
                              tuple(np.where(np.isin(x.get_variables(), variables))[0]))
    
    return factor(res_variables, res_distribution)

In [15]:
phi_4 = factor_marginalization(phi_3,['a'])
phi_4.get_distribution()

array([0.3, 0.7])

## Factor Graph